# Day 24: Never Tell Me The Odds

[*Advent of Code 2023 day 24*](https://adventofcode.com/2023/day/24) and [*solution megathread*](https://redd.it/18pnycy)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/24/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F24%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
example_input = '''19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3'''

In [5]:
from typing import Iterator, Tuple
import numpy as np 

type Coord = np.ndarray
type Vector = np.ndarray
type Hailstone = Tuple[Coord, Vector]

def parse_input(input: Iterator[str], ignore_z: bool=True) -> Iterator[Hailstone]:
    for line in input:
        coord_str, velocity_str = line.split(' @ ')
        coord_x, coord_y, coord_z = map(int, coord_str.split(', '))
        velocity_x, velocity_y, velocity_z = map(int, velocity_str.split(', '))
        if ignore_z:
            yield np.array([coord_x, coord_y]), np.array([velocity_x, velocity_y])
        else:
            yield np.array([coord_x, coord_y, coord_z]), np.array([velocity_x, velocity_y, velocity_z])

for line in parse_input(example_input.splitlines()):
    print(line)

(array([19, 13]), array([-2,  1]))
(array([18, 19]), array([-1, -1]))
(array([20, 25]), array([-2, -2]))
(array([12, 31]), array([-1, -2]))
(array([20, 19]), array([ 1, -5]))


$$
C_t = H(C_0, V, t) = C_0 + t*V
$$

So, at $t_A$ and $t_B$ respectively, the paths of hail $A$ and $B$ will cross:

$$
C_{tA} - C_{tB} = 0
$$

$$
C_{0A} + t_A*V_A - C_{0B} - t_B*V_B = 0
$$

$$
t_A*V_A - t_B*V_B = C_{0B} - C_{0A} 
$$

... i.e. determine scalars $t_A$ and $t_B$ to create a linear combination of $V_A$ and $V_B$ to equal $C_{0B} - C_{0A}$ (which one could have figured out intuitively...)


In [6]:
from typing import Iterable
from itertools import combinations

def find_crossings(hailstones: Iterable[Hailstone]) \
        -> Iterator[Tuple[Hailstone, float, Hailstone, float]]:
    for (C_0A, V_A), (C_0B, V_B) in combinations(hailstones, 2):
        V = np.array([V_A, -V_B]).transpose()
        C = C_0B - C_0A
        try:
            T = np.linalg.solve(V, C)
        except np.linalg.LinAlgError:
            continue
        yield (C_0A, V_A), T[0], (C_0B, V_B), T[1]

def find_future_crossings_in_bounds(hailstones: Iterable[Hailstone], b_min: int, b_max: int):
    def is_future(t: float) -> bool:
        return t > 0
    def calculate_position(C0: Coord, V: Vector, t: float) -> Coord:
        return C0 + t*V
    def in_bounds(b_min: int, b_max: int, hailstone: Hailstone, t: float) -> bool:
        Ct = calculate_position(*hailstone, t)
        return (b_min < Ct).all() and (Ct < b_max).all()
    
    for HA, TA, HB, TB in find_crossings(hailstones):
        if all(map(is_future, [TA, TB])) \
                and all(in_bounds(b_min, b_max, H, t) for H, t in [(HA, TA), (HB, TB)]):
            yield HA, HB, calculate_position(*HA, TA)

hailstones = parse_input(example_input.splitlines())
list(find_future_crossings_in_bounds(hailstones, 7, 27))
# hailstones = parse_input(downloaded['input'].splitlines())
# results = list(find_future_crossings_in_bounds(hailstones, 2e14, 4e14))

[((array([19, 13]), array([-2,  1])),
  (array([18, 19]), array([-1, -1])),
  array([14.33333333, 15.33333333])),
 ((array([19, 13]), array([-2,  1])),
  (array([20, 25]), array([-2, -2])),
  array([11.66666667, 16.66666667]))]

In [7]:
HTML(downloaded['part2'])